In [16]:
import pandas as pd
df=pd.read_csv('processed cleveland.csv',names=['age','sex','cp','trestbps','chol','fbs','restecg','thalach','exang',
                                               'oldpeak','slope','ca','thal','num'])
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0,6,0
1,67,1,4,160,286,0,2,108,1,1.5,2,3,3,2
2,67,1,4,120,229,0,2,129,1,2.6,2,2,7,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0,3,0


In [29]:
df=df.drop(['thal','ca'],axis=1)
df.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,num
0,63,1,1,145,233,1,2,150,0,2.3,3,0
1,67,1,4,160,286,0,2,108,1,1.5,2,2
2,67,1,4,120,229,0,2,129,1,2.6,2,1
3,37,1,3,130,250,0,0,187,0,3.5,3,0
4,41,0,2,130,204,0,2,172,0,1.4,1,0


In [30]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(df, test_size = 0.2)

In [31]:
train.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,num
214,52,1,4,112,230,0,0,160,0,0.0,1,1
277,39,0,3,138,220,0,0,152,0,0.0,2,0
193,62,0,4,138,294,1,0,106,0,1.9,2,2
186,42,1,3,120,240,1,0,194,0,0.8,3,0
23,58,1,3,132,224,0,2,173,0,3.2,1,3


In [32]:
test.head()

,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,num
271,66,1,4,160,228,0,2,138,0,2.3,1,0
276,66,0,3,146,278,0,2,152,0,0.0,2,0
264,61,1,4,138,166,0,2,125,1,3.6,2,4
43,59,1,3,150,212,1,0,157,0,1.6,1,0
26,58,0,3,120,340,0,0,172,0,0.0,1,0


In [45]:
from sklearn.tree import DecisionTreeClassifier
clf=DecisionTreeClassifier(max_leaf_nodes=15)
#clf=DecisionTreeClassifier()
clf=clf.fit(train.drop(['num'],axis=1).values,train['num'].values)

In [46]:
clf

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=15,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')

In [47]:
clf.feature_importances_

array([0.07197205, 0.03923733, 0.34021062, 0.08326448, 0.18373301,
       0.        , 0.        , 0.04023757, 0.        , 0.24134495,
       0.        ])

In [48]:
from sklearn import tree
with open("heart.dot","w") as f:
    f=tree.export_graphviz(clf,feature_names=train.drop(['num'],axis=1).columns,out_file=f)

In [43]:
predictions=clf.predict(test.drop(['num'],axis=1).values)

In [44]:
from sklearn.metrics import accuracy_score
accuracy_score(test['num'],predictions)

0.47540983606557374

In [49]:
from sklearn.ensemble import RandomForestClassifier
x_train=train.drop(['num'],axis=1).values
y_train=train['num'].values
x_test=test.drop(['num'],axis=1).values
y_test=test['num']

clf=RandomForestClassifier(n_estimators=1000,max_leaf_nodes=15)
clf=clf.fit(x_train,y_train)
predictions=clf.predict(x_test)
accuracy_score(y_test,predictions)

0.6065573770491803

In [51]:
from sklearn.ensemble import GradientBoostingClassifier
clf=GradientBoostingClassifier()
clf=clf.fit(x_train,y_train)
predictions=clf.predict(x_test)
accuracy_score(y_test,predictions)

0.47540983606557374

In [57]:
##stacking

clf=tree.DecisionTreeClassifier(max_leaf_nodes=25)
clf.fit(x_train,y_train)
prediction=clf.predict(x_train)

clf1=GradientBoostingClassifier(n_estimators=100,learning_rate=.1,max_depth=1,random_state=0)
clf1.fit(x_train,y_train)
prediction1=clf1.predict(x_train)

clf2=RandomForestClassifier(n_estimators=100)
clf2.fit(x_train,y_train)
prediction2=clf2.predict(x_train)

dataL2=pd.DataFrame(prediction,columns=['p1'])
dataL2=pd.concat([dataL2,pd.DataFrame(prediction1,columns=['p2']),pd.DataFrame(prediction2,columns=['p3'])],axis=1)
dataL2.head()

,p1,p2,p3
0,0,0,1
1,0,0,0
2,2,2,2
3,0,0,0
4,0,0,3


In [59]:
from sklearn.linear_model import LogisticRegression

l2x=dataL2.values
l2y=y_train

finalClassifier=LogisticRegression()
finalClassifier.fit(l2x,l2y)

#Test data generation
testPrediction=clf.predict(x_test)
testPrediction1=clf1.predict(x_test)
testPrediction2=clf2.predict(x_test)

testL2=pd.DataFrame(testPrediction,columns=['p1'])
testL2=pd.concat([testL2,pd.DataFrame(testPrediction1,columns=['p2']),pd.DataFrame(testPrediction2,columns=['p3'])],axis=1)
finalPred=finalClassifier.predict(testL2.values)
accuracy_score(y_test, finalPred)

C:\Users\gowri\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\gowri\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


0.5573770491803278